In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TextStreamer

hf_peft_repo = "LoRA_adapted_T5"
peft_config = PeftConfig.from_pretrained(hf_peft_repo)
model = AutoModelForSeq2SeqLM.from_pretrained(peft_config.base_model_name_or_path, return_dict=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)
streamer = TextStreamer(tokenizer)

# Load the finetuned Lora PEFT model
model = PeftModel.from_pretrained(model, hf_peft_repo)
model.eval()

[2023-07-13 13:50:07,047] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-13 13:50:20.426990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 13:50:20.720251: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-13 13:50:29.335391: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-13 13:50:29.335611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=512, out_features=512, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=512, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): P

In [2]:
from datasets import load_from_disk

test_set = load_from_disk('Training_set_IMDB/testing_dataset_sample/').with_format('torch')

In [10]:
test_set

Dataset({
    features: ['inputs', 'target'],
    num_rows: 4307
})

In [11]:


val_dataset = test_set.select(range(200))

In [12]:
val_dataset

Dataset({
    features: ['inputs', 'target'],
    num_rows: 200
})

In [36]:
def convert_to_features(example_batch, padding = "max_length",input_max = 64, output_max = 256):
    inputs = tokenizer.batch_encode_plus(example_batch["inputs"],is_split_into_words=True, padding='max_length', max_length=input_max, truncation=True)
    
    targets = tokenizer.batch_encode_plus(example_batch["target"],padding = "max_length", max_length=output_max,truncation = True)
    if padding == "max_length":
        targets["inputs_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in target] for target in targets["input_ids"]
        ]
    
    inputs["labels"] = targets['input_ids']
    return inputs

def evaluate_peft_model(sample):
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(),max_new_tokens = 200, top_p=0.9)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    _ = execution_accuracy(prediction, labels)
    
    return prediction, labels

def execution_accuracy(prediction, label):
    all_executions.append('1')
    try:
        cursor.execute(prediction)
        
        
        result_pred = cursor.fetchall()
        print(result_pred)
        
        cursor.execute(label)
        
        result_label = cursor.fetchall()
        print(result_label)
        if result_label == result_pred:
            accurate_executions.append("1")
    except:
            failed_executions.append("f")
    return None



In [37]:
import evaluate
import numpy as np
from tqdm import tqdm
import pandas as pd
import mysql.connector



connection = mysql.connector.connect(
    host="relational.fit.cvut.cz",
    user="guest",
    password="relational",
    database="imdb_ijs"
)
cursor = connection.cursor()



print("mapping")
tokenized_dataset = val_dataset.map(convert_to_features, batched=True, num_proc=4)
print("mapped, generating SQL")

all_executions = []
failed_executions = []
accurate_executions = []

predictions, references = [] , []
for sample in tokenized_dataset:
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)
    

cursor.close()
connection.close()


print(f"Execution rate: {round(100 - len(failed_executions)/len(all_executions)*100, 2)}%")
print(f"Execution accuracy: {round(len(accurate_executions)/len(all_executions)*100, 2)}%")


mapping
mapped, generating SQL
[]
[(50516, 'Julio César', 'Marmol', 1)]
[("Bernie's mother",)]
[("Bernie's mother",)]
[(43417, 'Akira', 'Kurosawa')]
[(43417, 'Akira', 'Kurosawa')]
[(40043, 'Anwar', 'Kawadri')]
[(40043, 'Anwar', 'Kawadri')]
[('Short',)]
[('Short',)]
[]
[]
[('Adult',)]
[('Adult',)]
[(None,)]
[(None,)]
[]
[(70093, 'Alberto Seixas', 'Santos')]
[(2591,)]
[(2591,)]
[(7.049999833106995,)]
[(7.049999833106995,)]
[]
[('Professor Marr',)]
[('Comedy',), ('Music',)]
[('Comedy',), ('Music',)]
[(18,)]
[(18,)]
[(9.300000190734863,)]
[(9.300000190734863,)]
[(0,)]
[(0,)]
[]
[(53662, 'Jacob Adrian', 'Mikkelsen', 5)]
[(6,)]
[(6,)]
[(1,)]
[(1,)]
[(53793, 'Chris (XIII)', 'Miller', 2)]
[(53793, 'Chris (XIII)', 'Miller', 2)]
[(110609, 'Fei teng de qun shan', 1976, None, 'Wei (I)', 'Li'), (300640, 'Si ge xiao huo ban', 1981, None, 'Wei (I)', 'Li')]
[(110609, 'Fei teng de qun shan', 1976, None, 'Wei (I)', 'Li'), (300640, 'Si ge xiao huo ban', 1981, None, 'Wei (I)', 'Li')]
[(None,)]
[(None,)]
[

[]
[(185134, 'Last Chance Detectives: Legend of the Desert Bigfoot, The', 1995, None), (279424, 'Road to Redemption', 2001, 4.9)]
[('Himself',)]
[('Himself',)]
[(3467, 'Phillip', 'Avalon', 1)]
[(3467, 'Phillip', 'Avalon', 1)]
[(63149, 'Harry A.', 'Pollard')]
[(63149, 'Harry A.', 'Pollard')]
[(42400, 'Nadejda', 'Koseva', 2)]
[(42400, 'Nadejda', 'Koseva', 2)]
[]
[]
[(14,)]
[(14,)]
[(None,)]
[(None,)]
[('Documentary',)]
[('Documentary',)]
[('Farnk',)]
[('Farnk',)]
[(626,)]
[(626,)]
[(None,)]
[(None,)]
[]
[(3182, 'Manabu', 'Asou')]
[(None,)]
[(None,)]
[(None,)]
[(None,)]
[(133834, 'Great Train Robbery, The', 1903, 7.5)]
[(133834, 'Great Train Robbery, The', 1903, 7.5)]
[(7252, 'Aedena', 2002, None), (24877, 'Avatars', 2001, None)]
[(24877, 'Avatars', 2001, None), (7252, 'Aedena', 2002, None)]
[('Short',)]
[('Short',)]
[(None,)]
[(None,)]
[]
[]
[('',)]
[('',)]
[(0,)]
[(0,)]
[(76725, 'Dag Jan', 1993, None, 'Jan', 'van den Berg'), (324248, 'Taming the Floods', 1999, None, 'Jan', 'van den Berg

[(230354, 'Never Leave Nevada', 1990, None)]
[(230354, 'Never Leave Nevada', 1990, None)]
[(None,)]
[(None,)]
[(None,)]
[(None,)]
[]
[]
[(243025, 'Opration Magali', 1953, None)]
[(243025, 'Opration Magali', 1953, None)]
[(32,)]
[(32,)]
[(11,)]
[(11,)]
Execution rate: 91.0%
Execution accuracy: 74.5%


In [ ]:
print(len(failed_executions))

In [31]:
print(val_dataset['target'][-1])

SELECT COUNT(distinct r.role) AS role_count FROM roles r JOIN movies m ON r.movie_id = m.id WHERE m.name = '"Gaffer, The"'


["SELECT a.first_name as first_name, a.last_name as last_name, r.role AS first_role FROM actors a INNER JOIN ( SELECT r.actor_id, MIN(m.year) AS min_movie_year FROM roles r INNER JOIN movies m ON r.movie_id = m.id GROUP BY r.actor_id ) t ON a.id = t.actor_id INNER JOIN roles r ON t.actor_id = r.actor_id INNER JOIN movies m ON r.movie_id = m.id AND t.min_movie_year = m.year WHERE a.first_name = 'Jack (VI)' AND a.last_name = 'Lewis'", "SELECT d.id AS director_id, d.first_name, d.last_name, COUNT(m.id) AS movie_count FROM directors d JOIN movies_directors md ON d.id = md.director_id JOIN movies m ON md.movie_id = m.id WHERE d.first_name = 'Julio César' AND d.last_name = 'Marmol' AND m.year = 1975 GROUP BY d.id, d.first_name, d.last_name"]
